# План домашних заданий


- Описание задачи и знакомство с данными
- Домашнее задание 1. Генерация и фильтрация признаков
- Домашнее задание 2. Прогноз времени и вида поломки машин, настройка ML-модели
- Дополнительное задание. Визуализация прогнозов, ошибок модели и важности признаков
- Домашнее задание 3. Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`
- Домашнее задание 4. Блендинг
- Домашнее задание 5. Парсинг внешних данных и оптимизация памяти
- Отправка финального решения на лидерборд на Kaggle

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
from phik import resources, report
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestClassifier
import shap

# <center id="part0"> Описание задачи

**Предыстория:**  мы работаем с каршеринговой компанией, которая управляет крупным автопарком машин. Наша цель — предотвратить длительные периоды простоя машин из-за поломок через своевременное обслуживание и ремонт.


**Идея для решения проблемы:** создать приоритизированный список обхода машин. Этот список поможет технической бригаде сосредоточиться на автомобилях, которые наиболее вероятно выйдут из строя в ближайшее время.


**Как компания решает задачу:**

* Собирает данные о поездках и состоянии машин до поломок.

* Нанимает Data Scientist, чтобы он смог использовать эти данные для анализа и прогнозирования характера поломок.

**Важный момент**: задачи этого специалиста (Data Scientist) предстоит выполнять вам.

## <center> Знакомство с данными

### Информация про машины с таргетом (основной датасет)

In [23]:
car_train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_train.csv')
car_train.sample(5)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
1411,S16201816J,Hyundai Solaris,economy,petrol,3.76,2011,1312,2015,78.49,another_bug
202,w-9263433r,Renault Sandero,standart,petrol,4.34,2012,26336,2020,58.14,another_bug
2118,x13220328w,Renault Sandero,standart,petrol,2.84,2013,42587,2018,32.56,gear_stick
1841,i19007345u,Mercedes-Benz E200,business,petrol,3.26,2013,43136,2016,42.17,engine_overheat
996,h90156916h,BMW 320i,business,petrol,3.86,2015,84170,2020,38.93,gear_stick


- `car_id` — идентификатор машины
- `model` / `car_type` / `fuel_type` — марка, класс и тип топлива машины
- `car_rating` / `riders` — общий рейтинг и общее число поездок к концу 2021 года
- `year_to_start` / `year_to_work` — год выпуска машины и год начала работы в автопарке
- `target_reg` — количество дней до поломки
- `target_class` — класс поломки (всего 9 видов)

### Информация про поездки

In [24]:
rides_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/rides_info.csv')
rides_info.sample(5)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
536068,j11528662N,l61851567T,k1Z,2020-03-19,6.22,40,475,42,155.0,1,693.197204,0,3.824714,-5.600
738944,X14255810v,z73740510r,I1N,2020-03-15,5.06,72,786,45,71.0,1,2867.312847,0,19.250412,-5.458
585114,p11345662X,p10312031l,P1I,2020-03-08,3.80,35,485,38,77.0,1,1230.392210,0,24.726463,-3.899
481180,z57638162h,i-1095441d,o1y,2020-02-07,5.68,46,730,52,87.0,1,2311.527635,0,7.633820,-0.000
561979,H13265758D,n19095522k,Z1d,2020-03-11,6.33,35,311,47,76.0,0,1072.289222,0,1.511421,20.042


- `user_id` / `car_id` / `ride_id` — идентификаторы водителя, машины, поездки соответственно
- `ride_date` / `rating` — дата поездки и рейтинг, поставленный водителем
- `ride_duration` / `distance` / `ride_cost` —  длительность (время),  пройденное расстояние, стоимость поездки
- `speed_avg` / `speed_max` — средняя и максимальная скорости поездки соответственно
- `stop_times` / `refueling` — количество остановок (паузы) и флаг: была ли дозаправка
- `user_ride_quality` — оценка манеры вождения водителя машины, определенная скоринговой ML-системой сервиса
- `deviation_normal` — общий показатель датчиков о состоянии машины, относительно эталонных показателей (нормы)

### Информация про водителей

In [25]:
driver_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/driver_info.csv')
driver_info.sample(5)

,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
5806,43,8.3,928,3.0,E66316409F,0,2019-3-14
10260,43,8.4,741,NaN,p15183175o,0,2019-5-4
2265,27,8.0,617,5.0,H22867778d,1,2020-7-22
12617,18,8.8,73,18.0,J21883011j,1,2019-12-6
4727,25,7.7,553,10.0,Q49004092W,1,2020-7-7


- `user_id` / `age` / `sex` — идентификатор, возраст и пол водителя, соответственно
- `user_rating` — общий рейтинг пользователя за все поездки к концу 2021 года
- `user_rides` — общее количество поездок к концу 2021 года
- `user_time_accident` — число инцидентов (это могли быть аварии/штрафы/эвакуация машины)  
- `first_ride_date` — дата первой поездки

### Информация про ремонт машин

In [26]:
fix_info = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/fix_info.csv')
fix_info.sample(5)

,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
37415,A48818516t,XP,2019-1-30 17:37,refuel,1.0,31
4707,K12580160n,NW,2019-12-23 7:19,reparking,1.0,28
52086,H-3434345i,FK,2020-6-15 2:0,reparking,1.0,20
89645,w-9263433r,DL,2019-4-5 2:59,repair,10.0,69
126047,D-1029823M,VW,2019-8-26 4:54,repair,5.0,17


- `worker_id` / `car_id` — идентификатор работника и машины
- `work_type` / `work_duration` — тип и длительность (в часах) проводимой работы
- `destroy_degree` — степень износа/поврежденности машины в случае поломки
- `fix_date` — время начала ремонта (снятия машины с линии)

# Домашнее задание 1. Генерация и фильтрация признаков

<center> <img src="https://ucarecdn.com/bf4c772d-b67a-42ae-a48b-cfd83910b0a2/" width=700>

<div class="alert alert-info">

**Цель блока** — сгенерировать признаки из дополнительных датасетов и добавить их к основному датасету, произвести фильтрацию признаков.

    
## Задание 1. Генерация признаков из дополнительных датасетов (6 баллов)

<div class="alert alert-info">

1. Возьмите датасет `rides_info` с информацией о поездках и проведите группировку по каждой машине отдельно.

2. Для каждой машины (то есть для каждого `car_id`) подсчитайте несколько признаков:
  - минимальный рейтинг за все поездки;
  - средний рейтинг за все поездки;
  - общее количество километров, которое машина проехала;
  - максимальная скорость;
  - общее количество поездок для каждой машины.

3. Добавьте минимум 3 признака на свой выбор.

4. Сделайте соединение таблиц — вспомните про методы соединения и выберите подходящий для этого случая.

5. Подключите информацию про водителей (`driver_info`) и про ремонт машин (`fix_info`). Добавьте минимум 3 любых признака с каждого датасета.


<h4> Критерии оценивания </h4>
    
- Добавлены 5 обязательных и минимум 3 любых признака из датасета `rides_info` — 2 балла.
- Добавлены минимум 3 любых признака из датасета `driver_info` — 2 балла.
- Добавлены минимум 3 любых признака из датасета `fix_info` — 2 балла.

</div>

<div class="alert alert-success">
    
<h4> Советы по Feature Engineering</h4>
    
- Начинайте с сырых данных.
- Используйте все доступные данные. Покрывайте признаками всю имеющуюся информацию в данных.
- Формулируйте предположения: от чего зависит таргет?
- Смотрите визуально на классы/ошибки и делайте предположения. Какие полезны?
- Помните, что слишком много признаков может быть вредно. Потом придется отфильтровывать.
    
<h4> Полезные источники </h4>
    
- Занятие про Feature Engineering.
- Занятие про Feature Selection.
- [Max Kuhn and Kjell Johnson. Feature Engineering and Selection: A Practical Approach for Predictive Models](http://www.feat.engineering/).

In [27]:
car_features = rides_info.groupby('car_id', as_index=False).agg(
    min_rating=('rating', 'min'), #минимальный рейтинг за все поездки
    avg_rating=('rating', 'mean'), #средний рейтинг за все поездки
    total_distance=('distance', 'sum'), #общее количество километров, которое машина проехала
    max_speed=('speed_max', 'max'), #максимальная скорость
    total_rides=('ride_id', 'count'), #общее количество поездок для каждой машины
    avg_deviation_normal=('deviation_normal', 'mean'), #cреднее отклонение от нормы по показателям состояния машины (чем выше среднее отклонение от нормы, тем вероятнее, что автомобиль сломается)
    avg_ride_duration=('ride_duration', 'mean'), #средняя длительность поездок (автомобили, которые часто используются для коротких поездок испытывают большее напряжение на двигатель и другие системы)
    total_stops=('stop_times', 'sum') #общее количество остановок (чем больше остановок, тем больше вероятность, что машина испытывает какие-то проблемы, требующие вмешательства или быстрее "изнашивается")
)

car_features.sample(5)

,car_id,min_rating,avg_rating,total_distance,max_speed,total_rides,avg_deviation_normal,avg_ride_duration,total_stops
3399,p87595615V,0.1,4.761149,1.294004e+07,165.263709,174,9.739052,1792.534483,156
509,G-7187279a,0.1,4.076322,1.555254e+07,199.749102,174,19.235109,1669.183908,141
2625,g-2087791I,0.1,3.855862,9.227756e+06,176.058775,174,14.185621,1113.063218,129
3904,v50135109z,0.1,4.478103,1.356316e+07,113.057472,174,-27.172638,1848.666667,145
3173,m76785981z,0.1,4.383448,1.109828e+07,185.647348,174,21.443247,1380.097701,143


In [28]:
merged_df = pd.merge(car_train, car_features, on='car_id', how='left')
#используем левое объединение, чтобы сохранить все строки из car_train
#если какая-то машина из car_train отсутствует в car_features, ее соответствующие признаки будут заполнены значением NaN

merged_df = pd.merge(merged_df, rides_info[['car_id', 'user_id']], on='car_id', how='left')
merged_df = pd.merge(merged_df, driver_info[['user_id', 'user_rating', 'user_rides', 'user_time_accident']], on='user_id', how='left')
merged_df = pd.merge(merged_df, fix_info[['car_id', 'work_type', 'work_duration', 'destroy_degree']], on='car_id', how='left')

merged_df.sample(5)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,...,avg_deviation_normal,avg_ride_duration,total_stops,user_id,user_rating,user_rides,user_time_accident,work_type,work_duration,destroy_degree
3908469,y-1806571O,Smart ForTwo,economy,petrol,5.24,2015,86243,2016,37.61,engine_fuel,...,35.441368,2281.706897,138,n18468040a,7.9,863,10.0,repair,65,10.0
9801556,K-2477266U,VW Polo VI,economy,petrol,5.04,2013,49733,2014,27.21,gear_stick,...,-12.875897,1367.413793,155,j22056698B,9.1,153,14.0,repair,2,3.5
2179523,n11802717C,Skoda Rapid,economy,petrol,4.30,2017,132832,2019,30.67,engine_fuel,...,17.375701,1873.408046,146,q14997645m,8.0,1621,7.0,repair,18,5.1
7645471,z91353693Z,Nissan Qashqai,standart,petrol,3.20,2013,50271,2016,23.64,break_bug,...,-8.031167,2170.580460,139,s21764921v,7.1,561,7.0,repair,20,5.4
10664444,K42844728e,Smart Coupe,economy,petrol,4.42,2014,58708,2019,61.07,engine_ignition,...,-15.783046,1521.936782,156,v11812675g,6.3,1557,9.0,reparking,40,5.0


## Задание 2. Применение методов фильтрации признаков (4 балла)

<center> <img src="https://ucarecdn.com/d1b4bc78-fd04-44fb-bdbf-0a63355b7384/" width=700>

1. Выберите и примените любые 3 (можно и больше) метода фильтрации, которые указаны выше или в ноутбуке по фильтрации из полезных ссылок.
2. Проинтерпретируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- Применены методы фильтрации — 1 балл за каждый метод, максимум 3 балла.
- Сделаны выводы на основе примененных методов —1 балл.
</div>

### Советы по Feature Selection

<div class="alert alert-info">

<h3><center>Зачем отбирать признаки</center></h3>
    
**Основные причины**:
    
- Главная причина: с увеличением количества признаков часто падает точность предсказания модели, а некоторые модели и вовсе перестают адекватно работать. Так происходит, если в данных большое количество мусорных фичей (почти не коррелирующих с таргетом).

- Если фичей очень много, то данные перестают помещаться в память и существенно увеличивают время обучения модели, особенно если мы тестируем несколько алгоритмов или ансамбль. Также важно учитывать, что платформы имеют ограничения на длительность одной сессии (в Kaggle — 12 часов) и лимиты по потребляемой памяти.

- Даже если точность не снижается, есть риск, что модель опирается на шумные фичи, что снизит стабильность прогноза на приватной выборке.


<b>Что удалить сразу:</b>
    
- константы;
- уникальные значения (в том числе в тесте; как правило, это ID-шники по типу `car_id`).
    
<b>Какие методы использовать:</b>
    
- `Линейная корреляция`.
- `Phik` — тоже корреляция, но на стероидах ([туториал на Medium](https://towardsdatascience.com/phik-k-get-familiar-with-the-latest-correlation-coefficient-9ba0032b37e7)).
- `Permutation importance` (из `scikit-learn`).
- `SHAP values` (из библиотеки `shap`).
- `CatBoost Feature Selection` (рекурсивные методы).
    

In [29]:
constant_columns = [column for column in merged_df.columns if len(merged_df[column].unique()) == 1]
constant_columns

['total_rides']

In [30]:
merged_df.drop(columns=['car_id', 'user_id', 'total_rides'], inplace=True)
cat_columns = merged_df.select_dtypes(['object']).columns
#merged_df[cat_columns] = merged_df[cat_columns].apply(lambda x: pd.factorize(x)[0])
merged_df = merged_df.dropna(axis = 1)

In [31]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13970460 entries, 0 to 13970459
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   model                 object 
 1   car_type              object 
 2   fuel_type             object 
 3   car_rating            float64
 4   year_to_start         int64  
 5   riders                int64  
 6   year_to_work          int64  
 7   target_reg            float64
 8   target_class          object 
 9   min_rating            float64
 10  avg_rating            float64
 11  total_distance        float64
 12  max_speed             float64
 13  avg_deviation_normal  float64
 14  avg_ride_duration     float64
 15  total_stops           int64  
 16  user_rating           float64
 17  user_rides            int64  
 18  work_type             object 
 19  work_duration         int64  
 20  destroy_degree        float64
dtypes: float64(10), int64(6), object(5)
memory usage: 2.3+ GB


In [43]:
cat_columns

Index(['model', 'car_type', 'fuel_type', 'target_class', 'work_type'], dtype='object')

# Домашнее задание 2. Прогноз времени и вида поломки машин. Настройка ML-модели

<div class="alert alert-info">

**Цель блока** — составить тренировочную и валидационную выборки, произвести обучение модели `CatBoostClassifier` на тренировочной части и оценить качество на валидационной.

##Обучение первой модели (5 баллов)

1.   Новый пункт
2.   Новый пункт



1. Классифицируйте признаки на типы (категориальные, числовые, таргеты).

2. Выделите в `X` только признаки, а в `Y` — таргет (для задачи классификации).

3. Сделайте разделение данных на *обучающую* и *валидационную* выборки (не забывайте про воспроизводимость результатов).

4. Создайте и обучите `CatBoostClassifier` модель (настраивать гиперпараметры сейчас не обязательно).

5. Проведите оценку вашей модели, используя метрику `accuracy`.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.

</div>

In [32]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [44]:
features2drop = merged_df.drop('target_class', axis=1) # то, что надо выбросить
targets = merged_df['target_class'] # таргеты
cat_features = cat_columns # категориальные признаки
num_features = merged_df.select_dtypes(['int64', 'float64']).columns # числовые признаки

print('Категориальные признаки:', len(cat_features), cat_features)
print('Числовые признаки:', len(num_features), num_features)
print('Целевые переменные', targets)

Категориальные признаки: 5 Index(['model', 'car_type', 'fuel_type', 'target_class', 'work_type'], dtype='object')
Числовые признаки: 16 Index(['car_rating', 'year_to_start', 'riders', 'year_to_work', 'target_reg',
       'min_rating', 'avg_rating', 'total_distance', 'max_speed',
       'avg_deviation_normal', 'avg_ride_duration', 'total_stops',
       'user_rating', 'user_rides', 'work_duration', 'destroy_degree'],
      dtype='object')
Целевые переменные 0            another_bug
1            another_bug
2            another_bug
3            another_bug
4            another_bug
                ...     
13970455    engine_check
13970456    engine_check
13970457    engine_check
13970458    engine_check
13970459    engine_check
Name: target_class, Length: 13970460, dtype: object


In [55]:
X = merged_df[num_features]
y = merged_df['target_class']

In [56]:
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.2)
X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid, test_size=0.5)

In [57]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

In [50]:
#merged_df[cat_columns] = merged_df[cat_columns].apply(lambda x: pd.factorize(x)[0])

In [58]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13970460 entries, 0 to 13970459
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   model                 int64  
 1   car_type              int64  
 2   fuel_type             int64  
 3   car_rating            float64
 4   year_to_start         int64  
 5   riders                int64  
 6   year_to_work          int64  
 7   target_reg            float64
 8   target_class          int64  
 9   min_rating            float64
 10  avg_rating            float64
 11  total_distance        float64
 12  max_speed             float64
 13  avg_deviation_normal  float64
 14  avg_ride_duration     float64
 15  total_stops           int64  
 16  user_rating           float64
 17  user_rides            int64  
 18  work_type             int64  
 19  work_duration         int64  
 20  destroy_degree        float64
dtypes: float64(10), int64(11)
memory usage: 2.3 GB


In [67]:
cb_mc = CatBoostClassifier(random_seed=43,
                           iterations=10,
                           eval_metric='Accuracy',
                           learning_rate=0.1,
                           )

cb_mc.fit(X_train, y_train, eval_set=(X_valid, y_valid))


0:	learn: 0.8823775	test: 0.8824634	best: 0.8824634 (0)	total: 12.3s	remaining: 1m 50s
1:	learn: 0.8823775	test: 0.8824634	best: 0.8824634 (0)	total: 35.9s	remaining: 2m 23s
2:	learn: 0.8990368	test: 0.8992753	best: 0.8992753 (2)	total: 1m 1s	remaining: 2m 24s
3:	learn: 0.8930288	test: 0.8932412	best: 0.8992753 (2)	total: 1m 29s	remaining: 2m 14s
4:	learn: 0.8968846	test: 0.8971294	best: 0.8992753 (2)	total: 2m 8s	remaining: 2m 8s
5:	learn: 0.9067747	test: 0.9068220	best: 0.9068220 (5)	total: 2m 43s	remaining: 1m 49s
6:	learn: 0.8973082	test: 0.8975596	best: 0.9068220 (5)	total: 3m 24s	remaining: 1m 27s
7:	learn: 0.9075912	test: 0.9077081	best: 0.9077081 (7)	total: 4m 2s	remaining: 1m
8:	learn: 0.9075893	test: 0.9076773	best: 0.9077081 (7)	total: 4m 36s	remaining: 30.8s
9:	learn: 0.9118665	test: 0.9119864	best: 0.9119864 (9)	total: 5m 15s	remaining: 0us

bestTest = 0.9119864342
bestIteration = 9



Точность (accuracy) модели досттаточно высокая, лучший результат на 9 итреации, точность - 0.912

# Дополнительное задание. Визуализация прогнозов, ошибок модели и важности признаков

<div class="alert alert-info">

**Цель блока** — визуализировать результаты прогнозирования, ошибки модели и важность признаков для лучшего понимания и анализа модели.

## Визуализация (5 баллов)

   
1. Визуализируйте важность признаков встроенным методом в `CatBoost`  (помните, что не стоит сильно доверять этому рейтингу важности, т. к. для сильно скоррелированных признаков важность делится пополам — оба признака могут улететь вниз).

2. Постройте `waterfall_plot` из библиотеки `SHAP` (подробнее во втором ноутбуке из полезных ссылок ниже).
    
3. Постройте `classification_report` из библиотеки `scikit-learn`.

4. Постройте и визуализируйте матрицу смежности (`confusion_matrix`), посмотрите, в каких классах модель больше всего ошибается.
    
5. Для каждого графика/примененного метода проинтерпретируйте результаты и сделайте выводы.


<h4> Критерии оценивания </h4>
    
- По 1 баллу за каждый корректно выполненный пункт.
    
</div>

<div class="alert alert-success">

Визуализация может помочь даже после того, как мы **уже обучили** какую-нибудь модель. Например:

- Понять, что мешает модели или чего не хватает, чтобы не допускать ошибки.
- Сделать выводы, как можно улучшить точность в последующих экспериментах.
- Визуализировать ошибки модели.
- Отсеять лишние признаки.
- Найти идеи для новых признаков.
- Все зависит от типа ваших данных.

<h3> Полезные источники </h3>
    
- Занятие про визуализацию и Seaborn.
- Занятие про продвинутую визуализацию и анализ ошибок модели.

</div>    

In [34]:
# YOUR CODE HERE

# Домашнее задание 3. Оптимизация. Тюнинг гиперпараметров с помощью `Optuna`

<div class="alert alert-info">

**Цель блока** — улучшить качество предсказания, произведя подбор гиперпараметров для модели с помощью `Optuna`.

## Подбор гиперпараметров (3 балла)

1. Напишите `objective` функцию и запустите `Optuna`.
2. Подберите гиперпараметры для `CatBoostClassifier` (минимум 3).
3. Обучите модель с новыми гиперпараметрами, сравните качество и сделайте выводы.

    
<h4> Критерии оценивания </h4>
    
- 1 балл за каждый корректно выполненный пункт.
    
</div>

In [35]:
# YOUR CODE HERE

### Общая информация по `Optuna`

#### Ключевые особенности <code>Optuna</code>



- Легковесность и универсальность — можно подбирать оптимальные параметры под любые функции и метрики.
- SOTA-алгоритмы, адаптированные для поиска гиперпараметров.
- Параллелизация и различные методы прунинга.
- Встроенная визуализация.
- Интеграция со множеством популярных библиотек (бустинги, sklearn, PyTorch, W&B и другие).

Разберем Optuna по частям, чтобы понять, как ее использовать.

#### <code>Study</code>: оптимизация, базирующаяся на <code>Objective</code> функции



В `Objective`-функцию нужно написать код подсчета метрики, которую возвращаем. `Objective` вызывается Optuna много раз для подбора лучших параметров.
```python
def objective(trial, ...):
    # calculate score...
    return score
```

<h4> <code>Trial</code> — одно выполнение <code>Objective</code>-функции</h4>

В `trial`-объекте мы передаем параметры для «перебора», используя для каждого типа свой метод. Например:

```python
# метод `suggest_float` показывает, что перебираем `float` значения, от 0 и до 1.5 границы
param = trial.suggest_float('param', 0, 1.5)

# Категориальное значение
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Целочисленное значение
depth = trial.suggest_int('depth', 5, 8)

# Равномерное распределение
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```

#### `Study` parameters

Инициализируем объект `study`, который начнет перебор и сохранит в себе историю результатов.
Если мы стараемся увеличить метрику, а не уменьшить ошибку, то используем `create_study(direction='maximize')`.
```python
study = optuna.create_study()
study.optimize(objective, n_trials=10)
```

<div class="alert alert-info">
    
В [`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) реализовано несколько методов (`sampler`) подбора параметров (в том числе классические):
* `GridSampler`
* `RandomSampler`
* `Tree-Structed Parzen Estimator` (`TPESampler` — самый популярный, дефолтный)
* `BruteForceSampler`
* [Другие 4 метода](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers) (можно написать собственный семплер)

### Советы по перебору параметров

- Иметь понимание важности параметров.
- Число `iterations` лучше взять с запасом и зафиксировать, при этом ограничив через `early_stopping_rounds`.
- Подсмотреть или чувствовать диапазоны и шаг значений.
- Исключить то, что перебирать не нужно (`random_seed` , `eval_metric`, `thread_count` и прочее).
- Используйте информацию с прошлых попыток.

# Домашнее задание 4. Блендинг

<div class="alert alert-info">

**Цель блока** — улучшить качество предсказания, объединив несколько моделей вместе методом блендинга.

## Задание 6. Блендинг (10 баллов)

1. Построить и обучить модели:
    - `CatBoostClassifier`,
    - `LightGBMClassifier (goss)`,
    - `XGBoostClassifier (dart)`,
    - `RandomForestClassifier`.
2. Сделать предсказания каждой моделью, оценить качество.
3. Реализовать блендинг двумя способами, оценить качество и сравнить с предыдущим пунктом:
    - `Hard Voting` —  метод, при котором делаем голосование всех моделей и выбираем самый популярный класс.
    - `Soft Voting` —  метод, при котором мы складываем вероятности предсказания всех моделей по каждому классу и потом выбираем самый класс с максимальной суммой.
    
<h4> Критерии оценивания </h4>
    
- Обучены 4 модели — 1 балл за каждую обученную модель, всего 4 балла.
- Для модели проведен подбор гиперпараметров — 0,5 балла за каждый параметр, всего 2 балла.
- Сделаны предсказания каждой модели и оценено качество — 1 балл.
- Реализован `Hard Voting` — 1 балл.
- Реализован `Soft Voting` — 1 балл.
- Сделаны выводы — 1 балл.
    
</div>

In [36]:
# YOUR CODE HERE

### Общая информация о блендинге

**Основная идея** — взять от каждого алгоритма лучшее и совместить несколько разных ML-моделей в одну.

Что это дает:
- Увеличивается обобщающая способность финальной модели и качество улучшается.
- Модель становится более стабильной, что позволяет не слететь на приватном лидерборде.

Особенно хорошо накидывает блендинг, если смешиваемые модели имеют разную природу — например, нейронные сети, kNN и решающие деревья. Они выучивают разные зависимости и хорошо дополняют друг друга.

</div>

# <center id="part6"> Отправка финального решения на лидерборд на Kaggle

<div class="alert alert-info">

**Цель блока** — сделать предсказание с помощью блендинга для тестовой части датасета, отправить результат [в соревнование на Kaggle](https://www.kaggle.com/competitions/competative-data-science-course-by-data-feeling/overview).

## Задание 8. Предсказание на тестовом датасете и отправка на Kaggle

1. Сделать предобработку для тестового датасета, присоединить к нему информацию из других датасетов и добавить признаки, которые генерировали для тренировочного датасета.
2. Сделать предсказания каждой моделью, которую хотите включить в ансамбль, но минимум их должно быть 3.
3. Сделать блендинг с помощью `Hard Voting` или `Soft Voting` на ваш выбор.
4. Сохранить результат предсказания в `csv` файл и отправить решение на Kaggle.
    
<h4> Критерии оценивания </h4>
    
- 0 баллов за задание, если итоговый скор на лидерборде меньше чем `0.9`.
- 1 балл за каждый выполненный пункт.

</div>

In [37]:
car_test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_test.csv')
car_test.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017


In [38]:
# YOUR CODE HERE

### Советы по отправке кода на Kaggle



Если пишете код на `Kaggle`, то для отправки решения:

1. Нажать Save Version.
2. Проверить, что в Advanced Settings выбрано «Всегда сохранять результаты ноутбука».
3. Выбрать тип сохранения Save & Run All (Commit).
4. Нажать кнопку Save.